In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"data/gemstone.csv")

In [4]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df = df.drop(labels=['id'],axis=1)

In [6]:
#indepdent and dependent feature

X= df.drop(labels=["price"],axis=1)
y = df[['price']]

In [7]:
#define which column should be ordinal-encoding and which should be scaled
categorical_col = X.select_dtypes(include='object').columns
numerical_col = X.select_dtypes(exclude='object').columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [22]:
#define the custom ranking for each ordinal variable
cut_categories = ['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_categories = ['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_categories = ['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']


In [23]:
from sklearn.impute import SimpleImputer        #handling missing values
from sklearn.preprocessing import StandardScaler    #handling feature scaling
from sklearn.preprocessing import OrdinalEncoder    #ordinal encoder
#pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [24]:
#numerical pipline

num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)


#categorical pipline

cat_pipeline = Pipeline(
    steps =[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_col),
    ('cat_pipeline',cat_pipeline,categorical_col)
])

In [25]:
#train test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.30,random_state=42)

In [26]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test  = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [27]:
X_train.head(3)

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.652424,0.058753,1.605394
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-0.381506,1.123070,-0.079246
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-1.415437,1.123070,-1.202339


In [28]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [29]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [30]:
regression.coef_

array([[ 6538.82211476,  -232.4061404 ,  -177.05602534, -2090.53215704,
         -556.05883268,   -44.19321465,   -10.72128479,  -181.811724  ,
          121.17907723]])

In [31]:
regression.intercept_

array([3976.8787389])

In [32]:
import numpy as np
def evaluate_model(true,predicated):
    mae = mean_absolute_error(true,predicated)
    mse = mean_squared_error(true,predicated)
    rmse = np.sqrt(mean_squared_error(true,predicated))
    r2_square= r2_score(true,predicated)
    return mae,mse,rmse,r2_square

In [33]:
#train multiple models

models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

model_list = []
r2_list = []


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    #make predictions
    y_pred = model.predict(X_test)
    
    mae,mse,rmse,r2_square = evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print("Modek Training Performance")
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score:",r2_square*100)

    r2_list.append(r2_square)

    print("="*35)
    print("\n")

LinearRegression
Modek Training Performance
RMSE: 1196.8248815972495
MAE: 723.6034569047443
R2 score: 91.1356057138532


Lasso
Modek Training Performance
RMSE: 1196.793686086361
MAE: 725.0550559822417
R2 score: 91.13606781270963


Ridge
Modek Training Performance
RMSE: 1196.830785634526
MAE: 723.637074937566
R2 score: 91.13551825604011


ElasticNet
Modek Training Performance
RMSE: 1632.5394144265958
MAE: 1135.0835947511694
R2 score: 83.5064099195437


